In [17]:
import numpy as np

periodic = True

from_edge = False
to_edge = True

axis = 0

a = np.array([3,4,7,2,5])
a1 = np.roll(a,1)


shape = list(a.shape)


if from_edge and not to_edge:
    change = -1
elif not from_edge and to_edge:
    change = 1
else:
    change = 0

shape[axis]+=change

diff = np.zeros(tuple(shape))



if to_edge and periodic:
    diff_slice = slice(-1)
else:
    diff_slice = slice(None)


if from_edge and periodic:
    # this needs to be done BEFORE rolling
    cut_slice = slice(-1)
else:
    cut_slice = slice(None)


diff[diff_slice]+=a[cut_slice]
diff[diff_slice]-=a1[cut_slice]


if to_edge and periodic:
    diff[-1] = diff[0]

if to_edge and not periodic:
    diff[-1] = np.nan # this can be incorrect if the kernel is asymmetric but OK for now



diff

array([-2.,  1.,  3., -5.,  3., -2.])

In [27]:
a = np.array([3,4,2,9])

a[1:4]

array([4, 2, 9])

In [40]:
import time

n_times = 10000
shape = (100,100)

A_obj = np.ones(shape,dtype="object")
A = np.full(shape,np.nan)

def timeit(f,name):
    start = time.time()
    f()
    end = time.time()
    print(f"{name}: {end-start}")

def obj_calc():
    for _ in range(n_times):
        A_obj*0



def float_calc():
    for _ in range(n_times):
        A*0



timeit(obj_calc,"object")

timeit(float_calc,"float")


object: 0.5840003490447998
float: 0.02509903907775879
